# HAAR CASCADES TO DETECT THINGS

*** 2001 Paul Viola and Michael Jones***
must read.
The simple idea behind the paper was that building a robust single step classifier is a computationally  intensive process.There is always a trade off between speed and acurracy in machine learning.*** SO THE IDEA IS TO BUILD A SIMPLE CLASSIFIER AND CASCADE THEM TOGETHER TO FORM UNIFIED CLASSIFIER THAT'S IS ROBUST.***
#2
HAAR FEATURES ARE SIMPLE SUMMATION AND DIFFERENCE OF PATCHES ACROSS THE IMAGES.



# INTEGRAL IMAGES

To find summation and difference of many different rectangular region within image is expensive process,to get rid of it we use concept of integral images.

that is:-if *** AP indicate sum of all elements in rectangle from a rectangle with A and P as opposite corners.***

now area of rectangle ABCD=AC-(AB+AD-AA)

*** WE CARE FOR THIS FORMULA BECAUSE EXTRACTING HAAR FEATURE INCLUDES COMPUTING THE AREAS OF A LARGE NUMBER OF RECTANGLE IN THE IMAGE AT MULTIPLE SCALES.
! The good thing about this approach
is that we don't have to recalculate anything. All the values for the areas on the right
hand side of this equation are already available. So we just use them to compute the
area of any given rectangle and extract the features.***


# DETECTING AND TRACKING FACES

#1
CascadeClassifier is used to load the xml file.*** The second argument
in this function specifies the jump in the scaling factor. As in, if we don't find an
image in the current scale, the next size to check will be, in our case, 1.3 times bigger
than the current size. The last parameter is a threshold that specifies the number of
adjacent rectangles needed to keep the current rectangle. It can be used to increase
the robustness of the face detector.***

In [1]:
import cv2
import numpy as np


cap=cv2.VideoCapture(0)
face_cascasde=cv2.CascadeClassifier('cascade\haarcascade_frontalface_alt.xml')

scale=0.5
while True:
    
    ret,frame=cap.read()
    
    frame = cv2.resize(frame, None, fx=scale,
                        fy=scale, interpolation=cv2.INTER_AREA)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    face_rect=face_cascasde.detectMultiScale(gray,1.3,5)
    
    for (x,y,w,h) in face_rect:
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
       
        
        
    cv2.imshow('face dectector',frame)
    
    c=cv2.waitKey(1)
    if c==27:
        break
        
cap.release()
cv2.destroyAllWindows()
                                    


# APPLYING FILTER ON FACE

e we know where the face is, we need to modify the coordinates a
bit to make sure the mask fits properly. This manipulation process is subjective
and depends on the mask in question. Different masks require different levels of
adjustments to make it look more natural. We extract the region-of-interest from the
input frame in the following line:
frame_roi = frame[y:y+h, x:x+w]

#2

we resize the input mask to make sure it fits in this region of interest 
*** THE INPUT MASK HAS A WHITE BACKGROUND.SO ONLY THE FILTER MUST BE PRESENT BEFORE OVERLAPING AND MAKING THE REMAINING PORTION TRANSPARENT***

#3
we create a mask by thresholding the skull image. Since the
background is white, we threshold the image so that any pixel with an intensity
value greater than 180 becomes 0, and everything else becomes 255

#4
frame region-of-interest is concerned, we need to black out everything in this mask
region. We can do that by simply using the inverse of the mask we just created. Once
we have the masked versions of the skull image and the input region-of-interest, we
just add them up to get the final image



In [2]:
# COODE

import cv2
import numpy as np

face_cascade=cv2.CascadeClassifier('cascade\haarcascade_frontalface_alt.xml')

face_mask=cv2.imread('hanniba.png') # to read the png mask

h_m,w_m=face_mask.shape[:2]

if face_cascade.empty():
    raise IOError('unable to load the face cascade classifier')
    
    
cap=cv2.VideoCapture(0)

scale=0.5

while True:
    
    ret,frame=cap.read()
    
    frame=cv2.resize(frame,fx=1.5,fy=1.5,interpolation=cv2.INTER_AREA)# TO RESIZE OUR SCRREN
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)# to convert into grayscale
    
    face=face_cascade.detectMultiScale(gray,1.3,5)
    
    
    for (x,y,w,h) in face:
        
        if h>0 and w>0:
            
            # adjust the h and w depending on the sizes 
            
            h,w=int(1.4*h),int(1*w)
            
            y-= 0.1*h
            
            # extract the reign of interest
            
            frame_roi=frame[y:y+h,x:x+w]
            
            
            face_mask_small=cv2.resize(face_mask,(w,h),interpolation=cv2.INTER_AREA)
            
            #CONVERT color image to grayscale and threshold it
            
            gray_mask=cv2.cvtColor(face_mask_small,cv2.COLOR_BGR2GRAY)
            
            ret,mask=cv2.threshold(gray_mask,180,255,cv2.THRESH_BINARY_INV)
            
            
            # CREATE A INVERSE MASK
            
            mask_inv=cv2.bitwise_not(mask)
            
            # use the mask to extract the face mask region of interest
            
            masked_face=cv2.bitwise_and(face_mask_small,face_mask_small,mask=mask)
            
            # use the inverse mask to get the remaining part of the image
            
            masked_frame=cv2.bitwise_and(frame_roi,frame_roi,mask=mask_inv)
            
            # add the two images to get the final output
            
            frame[y:y+h,x:x+w]=cv2.add(masked_face,maske_frame)
            
            
        cv2.imshow('face_filter',frame)
        
        c=cv2.waitKey(1)
        if c==27:
            break
            
            
cap.release()

cv2.destroyAllWindows()

    

AttributeError: 'NoneType' object has no attribute 'shape'

# DETECTING EYES



In [3]:
#CODE
import cv2
import numpy as np


cap=cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier('cascade\haarcascade_frontalface_alt.xml')
eye_cascade=cv2.CascadeClassifier('cascade\haarcascade_eye.xml')
while True:
    
    ret,frame=cap.read()
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    face=face_cascade.detectMultiScale(gray,1.3,5)
    
    for(x,y,w,h) in face: # to get coordinate of face
        
        roi_gray=gray[y:y+h,x:x+w]
        roi_color=frame[y:y+h,x:x+w]
        
        eye=eye_cascade.detectMultiScale(roi_gray)
        
        for (x,y,w,h) in eye:
            
            center=(int(x+w*0.5),int(y+h*0.5))
            radius=int(0.3*(w+h))
            color=(0,255,0)
            thick=3
            cv2.circle(roi_color,center,radius,color,thick)
            
    cv2.imshow('eye dectector',frame)
    
    c=cv2.waitKey(1)
    if c==27:
        break
        

cap.release()
cv2.destroyAllWindows()
        
        

# DETECTING THE PUPILS

we can use shape analysis to detect the pupils.
#2
We invert the input image
and then convert it into grayscale image as shown in the following line:
gray = cv2.cvtColor(~img, cv2.COLOR_BGR2GRAY)

#3
 Inverting
the image is helpful in our case because the pupil is black in color, and black
corresponds to a low pixel value. We then threshold the image to make sure that
there are only black and white pixels. Now, we have to find out the boundaries of all
the shapes
#4
. We can
use the function boundingRect to obtain the coordinates of the bounding rectangle. 
#5
We can use the function contourArea to compute the area of any contour in
the image. So we can use these conditions and filter out the shapes. After we do that,
we are left with two pupils in the image


In [5]:
#CODE
import math
import cv2
import numpy as np
img = cv2.imread('input.jpg')
scaling_factor = 0.7
img = cv2.resize(img, None, fx=scaling_factor,fy=scaling_factor, interpolation=cv2.INTER_AREA)
cv2.imshow('Input', img)
gray = cv2.cvtColor(~img, cv2.COLOR_BGR2GRAY)

ret, thresh_gray = cv2.threshold(gray, 220, 255,cv2.THRESH_BINARY)

contours, hierarchy = cv2.findContours(thresh_gray,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


for contour in contours:
    area = cv2.contourArea(contour)
    rect = cv2.boundingRect(contour)
    x, y, width, height = rect
    radius = 0.25 * (width + height)
    area_condition = (100 <= area <= 200)
    symmetry_condition = (abs(1 - float(width)/float(height))
                          <= 0.2)
    fill_condition = (abs(1 - (area / (math.pi * math.pow(radius,
                      2.0)))) <= 0.3)
    if area_condition and symmetry_condition and fill_condition:
        cv2.circle(img, (int(x + radius), int(y + radius)),int(1.3*radius), (0,255,0), -1)
cv2.imshow('Pupil Detector', img)
c = cv2.waitKey()
cv2.destroyAllWindows()


# thank you
